In [ ]:
# required to make plots show up
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from labdrivers.ni import bnc2110
from labdrivers.srs import sr830
from labdrivers.keithley import keithley2400

In [ ]:
# define some parameters
vgate_min = 20
vgate_max = 50
vgate_step = 0.5
sleep_time = 0.05

vbias_min = -10E-3
vbias_max = 10E-3
vbias_step = 0.5E-3

gate_lines = np.arange(vgate_min, vgate_max, vgate_step)
bias_lines = np.arange(vbias_min, vbias_max, vbias_step)

# make data frames w vbias as columns, vg as index
lockin_x_2d = pd.DataFrame(columns=bias_lines, index=gate_lines)
lockin_y_2d = pd.DataFrame(columns=bias_lines, index=gate_lines)

# for temporarily storing each 1d gate sweep
lockin_x = []
lockin_y = []

for vbias in bias_lines:
    
    # connect to the instruments
    dac = bnc2110('Dev1')
    lockin = sr830(8)
    gate_keithley = keithley2400(20)

    # configure the gate keithley
    gate_keithley.setMeasure('current')
    gate_keithley.setSourceDC('voltage')
    gate_keithley.setCompliance('current', 0.5E-6)

    # turn the outputs on
    gate_keithley.rampOutputOn(vgate_min, 0.5)
    time.sleep(3)

    # set s-d bias to vbias
    dac.setVoltageOutput('ao1', vbias)
    
    lockin_tuples = []

    # do the gate sweep
    for volts in gate_lines:

        # set the keithley output, then take a measurement
        gate_keithley.setSourceDC('voltage', volts)
        lockin_tuples.append(lockin.getSnapshot(1,2))
        
        # sleep so that the gate doesn't sweep too fast
        time.sleep(sleep_time)

    # unpack [(x1, y1), (x2, y2), (x3, y3), ...] 
    # to [x1, x2, x3, ...] and [y1, y2, y3, ...]
    lockin_x, lockin_y = zip(*lockin_tuples)
        
    lockin_x_2d[vbias] = lockin_x
    lockin_y_2d[vbias] = lockin_y

    # to keep track of where we are in the scan
    print('V_bias: {:.3e}'.format(vbias))
    
    # turn the keithley output off
    gate_keithley.rampOutputOff(vgate_max, 0.5)

# save the data to file
lockin_x_2d.to_csv('path/to/save x', index=True)
lockin_y_2d.to_csv('path/to/save y', index=True)